# Get context data on models
Getting this context data allows us to add extra context data to our template. It modifies what data is available in our template and can combine multiple sources of data

In [ ]:
def get_context_data(**kwargs):
    context = super().get_context_data(**kwargs)
    
    context.update({
        'key1': value1,
        "key2": value2
    })
    
    context['single_item'] = value
    return context

## Common use cases
1. Adding form instances 
2. Including user permissions
3. Adding counts and aggregations
4. Including related data
5. Adding configuration settings

Example:

In [ ]:
from django.contrib.auth.mixins import LoginRequiredMixin, UserPassesTestMixin
from django.views.generic import ListView
from django.utils import timezone
from django.db.models import Count
from datetime import timedelta

class Requests(UserPassesTestMixin, LoginRequiredMixin, ListView):
    model = MyModel
    template_name = "requests.html"
    context_object_name = "m101_requests"
    ordering = ['-request_date']
    paginate_by = 10
    
    def get_context_data(self, **kwargs):
        context = super().get_context_data(**kwargs)
        today = timezone.now().date()
        context.update({
            'total_requests': MyModel.objects.filter(
                request_date__date=today
            ).count(),
            
            'pending_requests': MyModel.objects.filter(
                request_status=MyModel.RequestStatus.PENDING
            ).count(),
            
            'completed_requests': MyModel.objects.filter(
                request_status=MyModel.RequestStatus.COMPLETED
            ),
            
            'user_groups': [group.name for group in self.request.user.groups.all()],
            
            'status_summary': MyModel.objects.values('request_status').annotate(
                total=Count('id')
            ),
            
            'title': 'Requests',
            'current_user': self.request.user,
        })
        return context

# Getting choices from DB model 
In order to display as a select field all the choices from the model, we can specify those choices in the context data and then just check them as key, value pairs in html

In [ ]:
from django.contrib.auth.mixins import LoginRequiredMixin, UserPassesTestMixin
from django.views.generic import ListView
from M101.models import M101Requests
from django.utils import timezone


class M101_requests(UserPassesTestMixin, LoginRequiredMixin, ListView):
    model = M101Requests
    template_name = "requests.html"
    context_object_name = "m101_requests"
    ordering = ['-request_date']
    paginate_by = 20
    
    def get_context_data(self, **kwargs):
        context = super().get_context_data(**kwargs)
        today = timezone.now().date()
        context.update({
            'total_requests': M101Requests.objects.filter(
                request_date__date=today
            ).count(),
            
            'pending_requests': M101Requests.objects.filter(
                request_status=M101Requests.RequestStatus.PENDING
            ).count(),
            
            'completed_requests': M101Requests.objects.filter(
                request_status=M101Requests.RequestStatus.COMPLETED
            ),
            
            'user_groups': [group.name for group in self.request.user.groups.all()],
            'status_summary': M101Requests.objects.values('request_status').annotate(
                total=Count('id')
            ),
            
            'title': 'M101 Requests',
            'current_user': self.request.user,
            'order_options': [
                {'value': '-request_date', 'label': 'Date'},
                {'value': 'request_status', 'label': 'Status'},
                {'value': 'quantity', 'label': 'Quantity'},
            ],
            'current_order': self.request.GET.get('order', '-request_date'),
            'status_filter': self.request.GET.get('status'),
            'request_status_choices': M101Requests.RequestStatus.choices,
            'warehouse_status_choices': M101Requests.WarehouseStatus.choices,
        })
        return context
    

In [ ]:
<div class="mb-3">
    <label for="requestStatus" class="form-label">Request Status</label>
    <select class="form-select" id="requestStatus" name="request_status">
        {% for value, label in request_status_choices %}
            <option value="{{ value }}">{{ label }}</option>
        {% endfor %}
    </select>
</div>
<div class="mb-3">
    <label for="warehouseStatus" class="form-label">Warehouse Status</label>
    <select class="form-select" id="warehouseStatus" name="warehouse_status">
        {% for value, label in warehouse_status_choices %}
            <option value="{{ value }}">{{ label }}</option>
        {% endfor %}
    </select>
</div>